In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import argparse
import sys
from numba import njit, prange
import time
import ipywidgets as widgets

In [22]:
def parse_arguments():
    parser = argparse.ArgumentParser(description="BZ Reaction Simulator")
    parser.add_argument("--grid_size", type=int, default=100, help="Size of the 2D grid")
    parser.add_argument("--time_steps", type=int, default=10000, help="Number of time steps")
    parser.add_argument("--dx", type=float, default=0.01, help="Spatial step size")
    parser.add_argument("--dt", type=float, default=0.0001, help="Time step size")
    parser.add_argument("--f", type=float, default=1.4, help="Oregonator model constant f")
    parser.add_argument("--q", type=float, default=0.002, help="Oregonator model constant q")
    parser.add_argument("--epsilon", type=float, default=0.02, help="Oregonator model constant epsilon")
    parser.add_argument("--D_u", type=float, default=1.0, help="Diffusion coefficient for U")
    parser.add_argument("--D_v", type=float, default=0.5, help="Diffusion coefficient for V")
    parser.add_argument("--save_animation", action="store_true", help="Save animation as MP4")
    parser.add_argument("--fps", type=int, default=30, help="Frames per second for saved animation")
    parser.add_argument("--interval", type=int, default=50, help="Interval between frames in milliseconds")
    parser.add_argument("--cmap_u", default="viridis", help="Colormap for U species")
    parser.add_argument("--cmap_v", default="plasma", help="Colormap for V species")
    parser.add_argument("--vmin", type=float, default=0, help="Minimum value for color scaling")
    parser.add_argument("--vmax", type=float, default=1, help="Maximum value for color scaling")

    # Check if we're in a Jupyter environment
    if 'ipykernel' in sys.modules:
        # If in Jupyter, use default values
        return parser.parse_args([])
    else:
        # If running as a script, parse command line arguments
        return parser.parse_args()


In [23]:
@njit(parallel=True)
def compute_laplacian(Z, dx):
    lap = np.zeros_like(Z)
    nx, ny = Z.shape
    for i in prange(nx):
        for j in prange(ny):
            
            lap[i, j] = (-4 * Z[i, j] +
                        Z[(i+1)%nx, j] +
                        Z[(i-1)%nx, j] +
                        Z[i, (j+1)%ny] +
                        Z[i, (j-1)%ny]) / (dx * dx)
    return lap


In [24]:
@njit(parallel=True)
def rk4_step(U, V, dt, dx, f, q, epsilon, D_u, D_v):
    # Compute k1
    lap_U = compute_laplacian(U, dx)
    lap_V = compute_laplacian(V, dx)
    r1_U = U - U * U * V + f * (1 - U)
    r1_V = epsilon * (-U * U * V + q * (1 - V))
    k1_U = dt * (D_u * lap_U + r1_U)
    k1_V = dt * (D_v * lap_V + r1_V)

    # Compute k2
    U1 = U + 0.5 * k1_U
    V1 = V + 0.5 * k1_V
    lap_U1 = compute_laplacian(U1, dx)
    lap_V1 = compute_laplacian(V1, dx)
    r2_U = U1 - U1 * U1 * V1 + f * (1 - U1)
    r2_V = epsilon * (-U1 * U1 * V1 + q * (1 - V1))
    k2_U = dt * (D_u * lap_U1 + r2_U)
    k2_V = dt * (D_v * lap_V1 + r2_V)

    # Compute k3
    U2 = U + 0.5 * k2_U
    V2 = V + 0.5 * k2_V
    lap_U2 = compute_laplacian(U2, dx)
    lap_V2 = compute_laplacian(V2, dx)
    r3_U = U2 - U2 * U2 * V2 + f * (1 - U2)
    r3_V = epsilon * (-U2 * U2 * V2 + q * (1 - V2))
    k3_U = dt * (D_u * lap_U2 + r3_U)
    k3_V = dt * (D_v * lap_V2 + r3_V)

    # Compute k4
    U3 = U + k3_U
    V3 = V + k3_V
    lap_U3 = compute_laplacian(U3, dx)
    lap_V3 = compute_laplacian(V3, dx)
    r4_U = U3 - U3 * U3 * V3 + f * (1 - U3)
    r4_V = epsilon * (-U3 * U3 * V3 + q * (1 - V3))
    k4_U = dt * (D_u * lap_U3 + r4_U)
    k4_V = dt * (D_v * lap_V3 + r4_V)

    # Combine to get next step
    U_next = U + (k1_U + 2.0 * k2_U + 2.0 * k3_U + k4_U) / 6.0
    V_next = V + (k1_V + 2.0 * k2_V + 2.0 * k3_V + k4_V) / 6.0

    return U_next, V_next

In [25]:
def initialize_concentrations(grid_size, U_steady, V_steady, perturbation=0.01):
    """
    Initialize U and V near steady-state with small random perturbations.
    """
    U = U_steady + perturbation * (np.random.rand(grid_size, grid_size) - 0.5)
    V = V_steady + perturbation * (np.random.rand(grid_size, grid_size) - 0.5)
    return U, V

In [26]:
def main():
    args = parse_arguments()

    # Extract parameters
    grid_size = args.grid_size
    time_steps = args.time_steps
    dx = args.dx
    dt = args.dt
    f = args.f
    q = args.q
    epsilon = args.epsilon
    D_u = args.D_u
    D_v = args.D_v
    save_animation = args.save_animation
    fps = args.fps
    interval = args.interval
    cmap_u = args.cmap_u
    cmap_v = args.cmap_v
    vmin = args.vmin
    vmax = args.vmax

    # Steady-state values (approximate)
    # For the Oregonator model, steady states can be found by solving the equations:
    # U - U^2 V + f (1 - U) = 0
    # -epsilon * (U^2 V - q (1 - V)) = 0
    # For simplicity, we use U = 1.0 and V = 1.0 / q
    U_steady = 1.0
    V_steady = 1.0 / q

    # Initialize concentration grids with small perturbations around steady state
    try:
        U, V = initialize_concentrations(grid_size, U_steady, V_steady)
    except MemoryError:
        print("Error: Grid size too large for available memory.")
        sys.exit(1)

    # Clip initial values to prevent negative concentrations
    U = np.clip(U, 0, vmax)
    V = np.clip(V, 0, vmax)

    # Prepare figure for animation
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    ax_u, ax_v = axes
    im_u = ax_u.imshow(U, cmap=cmap_u, vmin=vmin, vmax=vmax, animated=True)
    im_v = ax_v.imshow(V, cmap=cmap_v, vmin=vmin, vmax=vmax, animated=True)
    ax_u.set_title('Species U Concentration')
    ax_v.set_title('Species V Concentration')
    for ax in axes:
        ax.axis('off')

    # Add colorbars
    cbar_u = fig.colorbar(im_u, ax=ax_u, fraction=0.046, pad=0.04)
    cbar_v = fig.colorbar(im_v, ax=ax_v, fraction=0.046, pad=0.04)
    cbar_u.set_label('Concentration of U')
    cbar_v.set_label('Concentration of V')

    # Initialize lists to store frames
    ims = []

    # For performance tracking
    start_time = time.time()

    # Simulation loop
    for i in range(time_steps):
        # Runge-Kutta integration
        U, V = rk4_step(U, V, dt, dx, f, q, epsilon, D_u, D_v)

        # Clip values to prevent overflow and invalid operations
        U = np.clip(U, vmin, vmax)
        V = np.clip(V, vmin, vmax)

        # Update visualization every 10 time steps for smoother animation
        if i % 10 == 0:
            im_u.set_data(U)
            im_v.set_data(V)
            # Update titles with current time
            current_time = i * dt
            ax_u.set_title(f'Species U Concentration\nTime: {current_time:.2f} s')
            ax_v.set_title(f'Species V Concentration\nTime: {current_time:.2f} s')
            ims.append([im_u, im_v])

        # Optional: Print progress every 10%
        if i % (time_steps // 10) == 0 and i != 0:
            elapsed = time.time() - start_time
            print(f"Progress: {i / time_steps * 100:.0f}% | Elapsed Time: {elapsed:.2f} s")

    # Create animation
    ani = animation.ArtistAnimation(fig, ims, interval=interval, blit=True, repeat_delay=1000)

    # Save or show animation
    if save_animation:
        try:
            Writer = animation.writers['ffmpeg']
            writer = Writer(fps=fps, metadata=dict(artist='BZ Simulator'), bitrate=1800)
            ani.save('bz_reaction_enhanced.mp4', writer=writer)
            print("Animation saved as 'bz_reaction_enhanced.mp4'.")
        except Exception as e:
            print(f"Error saving animation: {e}")
            plt.show()
    else:
        plt.tight_layout()
        plt.show()

if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [--grid_size GRID_SIZE]
                             [--time_steps TIME_STEPS] [--dx DX] [--dt DT]
                             [--f F] [--q Q] [--epsilon EPSILON] [--D_u D_U]
                             [--D_v D_V] [--save_animation] [--fps FPS]
                             [--interval INTERVAL] [--cmap_u CMAP_U]
                             [--cmap_v CMAP_V] [--vmin VMIN] [--vmax VMAX]
ipykernel_launcher.py: error: argument --f: invalid float value: '/Users/san./Library/Jupyter/runtime/kernel-v2-2435hQkfVqdUep5B.json'


SystemExit: 2